In [62]:
%pip install -qU langchain-ollama
%pip install -qU langchain-community
%pip install -qU sentence-transformers

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\python\projects\langchain\fun_agent_env\scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\python\projects\langchain\fun_agent_env\scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\python\projects\langchain\fun_agent_env\scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3",
    temperature=0.4,
)

ModuleNotFoundError: No module named 'langchain_ollama'

In [64]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that answers questions related to {subject}. Answer the user question.",
        ),
        ("human", "{question}"),
    ]
)

chain = prompt | llm | StrOutputParser()
answer = chain.invoke(
    {
        "subject": "AI",
        "question": "Give me Ideas of a graduation project.",
    }
)
print(answer)

Congratulations on your upcoming graduation! I'd be happy to help you come up with some ideas for your graduation project, especially if it's related to Artificial Intelligence (AI). Here are a few potential concepts to get you started:

1. **Chatbot for Customer Service**: Design and develop a chatbot that can assist customers with common queries or issues. You could integrate this with a popular platform like Facebook Messenger or WhatsApp.
2. **Image Classification using Deep Learning**: Create a project that uses convolutional neural networks (CNNs) to classify images into different categories, such as objects, scenes, or emotions.
3. **Natural Language Processing (NLP) for Sentiment Analysis**: Develop an NLP-based system that can analyze text data and determine the sentiment (positive, negative, neutral) of a given piece of writing.
4. **Recommendation System using Collaborative Filtering**: Design a recommendation engine that suggests products or services based on user behavior 

In [7]:
!cd

C:\Python\projects\LangChain\RAG


In [23]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub
from sentence_transformers import SentenceTransformer
import faiss

In [26]:
pdf_path = "Doc.pdf"
loader = PyPDFLoader(file_path=pdf_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(
    chunk_size=1000, chunk_overlap=30, separator="\n"
)
docs = text_splitter.split_documents(documents=documents)
str_docs = []
for i,doc in enumerate(docs):
    str_docs.append(str(docs[i]))
    
model = SentenceTransformer('all-MiniLM-L6-v2') # Loading our embedding model.

# Generate embeddings for the sentences
embeddings = model.encode(str_docs)
print(f"Embeddings shape: {embeddings.shape}")

# Dimensions of our embeddings
d = embeddings.shape[1]

# Creating an index for our dense vectors
index = faiss.IndexFlatL2(d)  # Using L2 (Euclidean) distance

# Adding the embeddings to the index
index.add(embeddings)

Embeddings shape: (191, 384)


In [27]:
from functools import cache
from typing import Iterator, Mapping, TypeVar, cast

import torch
from sentence_transformers import SentenceTransformer

T = TypeVar("T")

EMBEDDING_MODEL = "all-MiniLM-L6-v2"


@cache
def _model():
    model = SentenceTransformer(EMBEDDING_MODEL)
    return model

In [51]:
class VectorMap(Mapping[str, T]):
    """
    A dictionary that uses sentence-transformers to do vector comparison on
    keys
    """

    def __init__(self, data, threshold: float = 0.7):
        self._data: list[T] = data
        self._model = _model()
        self._embeddings = self._model.encode(self._data, convert_to_tensor=True)
        self._length = len(data)
        self._threshold = threshold

    def __getitem__(self, key) -> T:
        out = []
        top_k = 4
        query_embedding = self._model.encode(key, convert_to_tensor=True)
        similarity_scores = self._model.similarity(query_embedding, self._embeddings)[0]
        scores, indices = torch.topk(similarity_scores, k=top_k)
        for score, index in zip(scores, indices):
            if score > self._threshold:
                out.append(self._data[index])
        return out

    def __contains__(self, key) -> bool:
        query_embedding = self._model.encode(key, convert_to_tensor=True)
        scores = self._model.similarity_scores(query_embedding, self._embeddings)
        return any(score > self._threshold for score in scores)

    def __iter__(self) -> Iterator[str]:
        return iter(self._data)

    def __len__(self):
        return self._length

    def __repr__(self):
        return "VectorMap()"

In [52]:
vector_map = VectorMap(str_docs, threshold=0)

print(vector_map["This is just a test"])

['page_content=\'parameters, offering a balance between performance and efficiency. While smaller than\nGPT-3.5 and the rumored size of GPT-4, it still demonstrates impressive capabilities.\nOpen-Sour ce: Released under an open-source license, Llama v2 allows anyone to access,\nstudy, and modify the underlying code. This fosters collaboration and innovation in the\nfield of LLMs.\nFocus on Safety:  Meta emphasizes safety and factual accuracy with Llama v2. This makes it\na potentially strong candidate for applications where trust and responsible AI are crucial,\nsuch as mental health chatbots or educational tools.\nIn [ ]:while True:\n    message = input()\n    if message == \'Done\':\n        break\n    print(f"User Message: {message}")\n    print(f"LLM Responce: {chain.run(message)}") # This is an Arabic test for GPT-4\' metadata={\'source\': \'Doc.pdf\', \'page\': 22}', "page_content='wasn't trained on. A continuously decreasing loss on validation data indicates the model is\nlearni

In [60]:
vector_map["What is embeddings"][2]

"page_content='data in a high-dimensional space. Imagine them as unique fingerprints for your data points!\nHere are some key concepts to grasp embedding models:\nInput and Output:  They take raw, unprocessed data (think a sentence or an image) as\ninput and generate a vector of numbers (the embedding) as output.\nDimensionality:  The number of dimensions in this embedding vector determines the level\nof detail it can capture. Higher dimensions allow for more nuanced representations, but\nalso require more computational resources.\nSimilar ity: The magic lies in how embeddings represent relationships. Similar data points,\nlike sentences with similar meanings, will have embeddings closer together in the vector\nspace. Conversely, dissimilar data points will be further apart. This allows for efficient\nsimilarity searches – finding data points with characteristics close to a given query.\nPre-trained vs. Fine-tuned:  Embedding models can be:' metadata={'source': 'Doc.pdf', 'page': 39}"

In [61]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
retrieval_qa_chat_prompt

c:\python\projects\langchain\fun_agent_env\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMes

In [66]:
chain = retrieval_qa_chat_prompt | llm | StrOutputParser()
answer = chain.invoke(
    {
        "input": "Explain the context given to you!",
        "context": vector_map["What is RAG"],
    }
)
print(answer)

The provided context appears to be a collection of text snippets, likely from different documents or files. Here's what I've gathered:

1. The first snippet seems to be an error message from Jupyter Notebook, indicating that the kernel crashed while executing code in the current cell or a previous cell.
2. The second snippet is a passage about LLaMA v2, which appears to be an open-source large language model developed by Meta AI. It mentions the model's size, performance, and potential applications.
3. The third snippet discusses LangChain, an open-source framework for developing large language models (LLMs). It highlights the framework's modular design, core concepts, and benefits.

It seems that these snippets might be related to a project or research involving natural language processing, machine learning, or AI. However, I'm not sure what specific task or question is being asked, as there doesn't appear to be a clear query or prompt.

If you could provide more context or clarify th